In [1]:
# 패키지 설치
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 33.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 17.9 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=118a2ba3b150040b9f64ceec90c326b81dbcee8385b44bd1bc8f256d25b74c96
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


### 말뭉치 다운로드 및 전처리

오픈소스 파이썬 패키지 코포라(Korpora)를 활용해 BPE 수행 대상 말뭉치를 내려받고 전처리한다. 실습용 말뭉치는 박은정 님이 공개하신 Naver Sentiment Movie Corpus(NSMC)를 사용할 것.


In [3]:
from Korpora import Korpora

# 데이터를 내려받아 `nsmc`라는 변수로 읽어들이기
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 92.8MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 36.1MB/s]                            


In [4]:
import os

# 읽어들인 것 저장
# NSMC에 포함된 영화 리뷰들을 순수 텍스트 형태로 저장해두게 된다.
def write_lines(path, lines) :
  with open(path, 'w', encoding='utf-8') as f :
    for line in lines :
      f.write(f'{line}\n')

write_lines("./train.txt", nsmc.train.get_all_texts())
write_lines("./test.txt", nsmc.test.get_all_texts())

In [5]:
!head train.txt

아 더빙.. 진짜 짜증나네요 목소리
흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
너무재밓었다그래서보는것을추천한다
교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
원작의 긴장감을 제대로 살려내지못했다.
별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네
액션이 없는데도 재미 있는 몇안되는 영화
왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?


In [6]:
!head test.txt

굳 ㅋ
GDNTOPCLASSINTHECLUB
뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
지루하지는 않은데 완전 막장임... 돈주고 보기에는....
3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
음악이 주가 된, 최고의 음악영화
진정한 쓰레기
마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다
갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다
이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..


### GPT 토크나이저 구축

GPT 계열 모델이 사용하는 토크나이저는 Byte-level Byte Pair Encoding(BBPE). 우선 어휘집합 구축 결과를 저장해둘 디렉토리를 자신의 구글 드라이브 계정 내 `내 드라이브/nlpbook/bbpe`로 만들어 두고 저장.



In [7]:
import os

os.makedirs("/content/drive/MyDrive/중앙수업/nlpbook/bbpe", exist_ok=True)

In [9]:
# 아래 수행시 nsmc 데이터를 가지고 BBPE 어휘집합을 구축한다
from tokenizers import ByteLevelBPETokenizer

bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    
    # 학습 말뭉치를 리스트 형태로 넣기
    files=["/content/train.txt", "/content/test.txt"],

    # 어휘집합 크기 조절
    vocab_size = 10000,

    # 특수 토큰 추가
    special_tokens=["[PAD]"]
)
bytebpe_tokenizer.save_model("/content/drive/MyDrive/중앙수업/nlpbook/bbpe")

# >>
# ['/content/drive/MyDrive/중앙수업/nlpbook/bbpe/vocab.json',
#  '/content/drive/MyDrive/중앙수업/nlpbook/bbpe/merges.txt']

['/content/drive/MyDrive/중앙수업/nlpbook/bbpe/vocab.json',
 '/content/drive/MyDrive/중앙수업/nlpbook/bbpe/merges.txt']

In [10]:
# 바이트 레벨 BPE의 어휘 집합
!cat /content/drive/MyDrive/중앙수업/nlpbook/bbpe/vocab.json

{"[PAD]":0,"!":1,"\"":2,"#":3,"$":4,"%":5,"&":6,"'":7,"(":8,")":9,"*":10,"+":11,",":12,"-":13,".":14,"/":15,"0":16,"1":17,"2":18,"3":19,"4":20,"5":21,"6":22,"7":23,"8":24,"9":25,":":26,";":27,"<":28,"=":29,">":30,"?":31,"@":32,"A":33,"B":34,"C":35,"D":36,"E":37,"F":38,"G":39,"H":40,"I":41,"J":42,"K":43,"L":44,"M":45,"N":46,"O":47,"P":48,"Q":49,"R":50,"S":51,"T":52,"U":53,"V":54,"W":55,"X":56,"Y":57,"Z":58,"[":59,"\\":60,"]":61,"^":62,"_":63,"`":64,"a":65,"b":66,"c":67,"d":68,"e":69,"f":70,"g":71,"h":72,"i":73,"j":74,"k":75,"l":76,"m":77,"n":78,"o":79,"p":80,"q":81,"r":82,"s":83,"t":84,"u":85,"v":86,"w":87,"x":88,"y":89,"z":90,"{":91,"|":92,"}":93,"~":94,"¡":95,"¢":96,"£":97,"¤":98,"¥":99,"¦":100,"§":101,"¨":102,"©":103,"ª":104,"«":105,"¬":106,"®":107,"¯":108,"°":109,"±":110,"²":111,"³":112,"´":113,"µ":114,"¶":115,"·":116,"¸":117,"¹":118,"º":119,"»":120,"¼":121,"½":122,"¾":123,"¿":124,"À":125,"Á":126,"Â":127,"Ã":128,"Ä":129,"Å":130,"Æ":131,"Ç":132,"È":133,"É":134,"Ê":135,"Ë":136,"Ì":137

In [11]:
# 바이그램 쌍의 병합 우선순위
!cat /content/drive/MyDrive/중앙수업/nlpbook/bbpe/merges.txt

#version: 0.2 - Trained by `huggingface/tokenizers`
Ġ ì
Ġ ë
ì Ŀ
ë ĭ
í ķ
ê °
. .
ìĿ ´
ëĭ ¤
ë Ĭ
ì Ĺ
ê ³
ì §
ëĬ Ķ
ì ŀ
ë §
í Ļ
ê³ ł
ì ł
íĻ Ķ
ĺ ģ
Ġ ê
ë ı
ì ķ
ã ħ
ĺģ íĻĶ
ì ļ
ì§ Ģ
íķ ĺ
ê° Ģ
ë Ĥ
ê ²
ì Ħ
Ġì ŀ
¬ ë
ê ¸
Ġì ķ
ëı Ħ
Ġ í
ë ĵ
ë ¦
ìĹ Ĳ
Ġì Ŀ
íķ ľ
Ġì ĺģíĻĶ
Ī ë
³ ´
ì ĭ
ĸ ´
ìĿ ĺ
ê¸ °
ãħ ĭ
Ġì Ĺ
ìĿ Ģ
ë ¡
ë į
ìĿ Ħ
Ŀ ¼
ëĤ ĺ
ê² Į
ĠìĿ ´
ìĦ ľ
Ġë §
ìļ Ķ
ì Ĭ
ì ĸ´
ë¡ ľ
Ġë Ĥ
ë§ Į
ë Ŀ¼
ë¦ ¬
Ġì ł
· ¸
ëĭ Ī
ëĵ ¤
ë ¥
ê ±
ìķ Ħ
ë ł
.. .
ë ©
¬ ´
ì ľ
Ġ ê°
ìĿ ¸
ãħĭ ãħĭ
¯ ¸
ëį °
Ġì §
ë Ħ
Ġìķ Ħ
Į Ģ
ë Ł
ì ĺ
ê µ
íķ ´
Ġë ³´
ë© ´
ì ĥ
ì ĺģíĻĶ
Ġì ĭ
Ġê ·¸
ê ¹
ë °
Ġë ª
ìł Ĳ
ìĭ ľ
Ī ĺ
Ġë ĭ
ë ³´
ìĿ Į
ìĬ ¤
£ ¼
ë ŀ
Ġë °
ìľ ¼
ëĦ ¤
Ġìŀ ¬ë
ë¥ ¼
ë§ Ĳ
Ġì ¢
! !
ë ¶
ì ¤
Ġì Ĥ
ê± °
ì Ĥ
ì Ļ
ë ĮĢ
ëĭĪ ëĭ¤
Īë ¬´
ìŀ Ĳ
ëĬĶ ëį°
ì Ľ
Ġë ³
ìł ķ
Ġë Ħ
ë§ Ī
ê¹ Į
ì ²
ìĹ Ĩ
ĠìĹ Ĩ
ìĹ Ī
ĠëĤ ĺ
Ġ íķĺ
ìļ °
Ġë ´
ì ¹
ìķ ¼
Ġì¢ ĭ
ì £¼
ì§ Ħ
Ġëĭ ¤
ì Īĺ
í ĸ
ë ³
ë ²
ìł ģ
µ ľ
ìŀ ¥
ìŀ Ī
ìŀ ĳ
ìł Ħ
ìĥ ģ
ë ª
.. ..
Ġì ĥ
Ġìł ķ
ì§ ľ
ì Ĩ
° Į
ìľ¼ ë¡ľ
Ġê ²
Ġìŀ Ī
ì§Ģ ë§Į
í ĺ
ê° Ħ
ĠìĹ °
íķĺ ê³ł
Ġì Ļ
¬ë ŀ
ê³ ¼
Ĳ ëı
ìĺ ¤
Ġì Ĭ
Ġë ĵ
ëĤ ´
Ġê ¸
ı ī
ãħ ł
ĠëĦ Īë¬´
ëŁ °
ë ħ
Ġì ĸ´
Ġì ĺ

### BERT 토크나이저 구축

BERT는 워드피스(wordpiece) 토크나이저를 사용한다. 우선 어휘집합 구축 결과를 저장해둘 디렉토리를 자신의 구글 드라이브 계정 내 `내 드라이브/nlpbook/bbpe`로 만들어 저장.

In [12]:
import os

os.makedirs("/content/drive/MyDrive/중앙수업/nlpbook/wordpiece", exist_ok=True)

In [13]:
# 아래 수행시 BERT 모델이 사용하는 워드피스 어휘집합을 구축가능
from tokenizers import BertWordPieceTokenizer

wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size=10000,
)

wordpiece_tokenizer.save_model("/content/drive/MyDrive/중앙수업/nlpbook/wordpiece")

['/content/drive/MyDrive/중앙수업/nlpbook/wordpiece/vocab.txt']

In [14]:
# 워드피스 어휘집합인 vocab 생성된 것 확인
!head /content/drive/MyDrive/중앙수업/nlpbook/wordpiece/vocab.txt

[PAD]
[UNK]
[CLS]
[SEP]
[MASK]
!
"
%
&
'


### GPT 입력값 만들기

GPT 모델 입력값을 만들려면 Byte-level Byte Pair Encoding 어휘집합 구축 결과(`vocab.json`, `merges.txt`)가 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 있어야한다. 다음을 수행해 이미 만들어 놓은 BBPE 어휘집합을 포함한 GPT 토크나이저를 `tokenizer_gpt`라는 변수로 쓰겠다 선언.

In [15]:
from transformers import GPT2Tokenizer

# 해당 위치에 위 실습에서 만들어놓은 바이트 기준 BPE 어휘집합(vocab.json)과
# 바이그램 쌍의 병합 우선순위(merge.txt)가 있어야 실행 가능
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/중앙수업/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

file /content/drive/MyDrive/중앙수업/nlpbook/bbpe/config.json not found


In [16]:
# 예시 문장(sentences)를 바이트 수준 BPE 토크나이저로 토큰화한 모습
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]

tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

In [17]:
# 모델에 실제 입력값은 아래와 같이 만들게 된다
batch_inputs = tokenizer_gpt(
    sentences,

    # 문장의 최대 길이에 맞춰 패딩하겠다.
    padding="max_length",

    # 문장의 토큰 기준 최대 길이
    max_length=12,

    # 문장 잘림 허용 옵션
    truncation=True,
)

batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

위 코드 실행시 두 가지의 입력값(input_ids)와 일반 토큰이 자리한 곳(1)과 패딩토큰이 자리한 곳(0)을 구분해 알려주는 mask(attention_mask)가 만들어진다.

In [18]:
# input_ids는 토큰화된 결과를 가지고 각 토큰들을 인덱스로 바꾼것이다.
# 어휘집합(vocab.json)을 확인해보면 각 어휘가 순서대로 나열된 모습을 확인할 수 있는데, 그 순서가 인덱스가 된다
# 이와같이 각 토큰을 인덱스로 변환하는 과정은 '인덱싱(indexing)'이라 불린다.
batch_inputs['input_ids']

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [19]:
# 값을 자세히보면 예시로 넣었던 세 문장의 길이가 모두 12로 맞춰진 것을 볼 수 있다
# 이는 max_length = 12 를 해줬기 때문
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

In [20]:
batch_inputs

{'input_ids': [[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0], [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292], [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]}

### BERT 입력값 만들기
이번엔 BERT 모델의 입력값을 만들어보자. 바로 아래 코드를 실행하면 BERT 모델이 사용하는 토크나이저를 초기화할 수 있는데. 이때 당연히 자신의 구글 드라이브 경로에 BERT용 워드피스 어휘 집합(vocab.txt)이 있어야 한다.

In [21]:
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained("/content/drive/MyDrive/중앙수업/nlpbook/wordpiece", do_lower_case=False)

file /content/drive/MyDrive/중앙수업/nlpbook/wordpiece/config.json not found


In [22]:
# 예시 문장(sentences)를 바이트 수준 BPE 토크나이저로 토큰화한 모습
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]

tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

In [23]:
batch_inputs = tokenizer_bert(
    sentences,

    # 문장의 최대 길이에 맞춰 패딩하겠다.
    padding="max_length",

    # 문장의 토큰 기준 최대 길이
    max_length=12,

    # 문장 잘림 허용 옵션
    truncation=True,
)

batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [24]:
batch_inputs['input_ids']

[[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1257, 16, 3],
 [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [25]:
batch_inputs['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [26]:
batch_inputs

{'input_ids': [[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0], [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1257, 16, 3], [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

`token_type_ids`라는 입력값은 세그먼트(segment)에 해당하는 것으로 현재 실습에서는 문장을 하나씩 넣었기 때문에 모두 0으로 처리된 것이라 함.

token_type_ids는 전체 입력(input_ids)의 어느부분이 첫 번째 문장이고, 어느것이 두 번째 문장인지를 모델에 알려주는 값이라 한다.

일반적으로, 토큰화 완료된 입력에 token_type_ids가 있는지 여부에 대해 걱정할 필요는.. 없다고함. 토크나이저와 모델 모두에 동일한 체크포인트(checkpoint)를 사용하는 한, 토크나이저는 모델에 무엇을 제공해야 하는지 알고 있기 때문에

<sub>예시</sub> <br>
```python
# 데이터셋 전처리를 위해 우선적으로 텍스트를 모델이 이해할 수 있는 숫자로 변환하는 과정 = 전처리
# 이는 토크나이저가 담당
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

# 하지만 두 개의 시퀸스를 모델에 바로 전달(각각의 문장을 모델에 별도로 매개변수로 전달)
# 하게 된다면 두 문장이 의역인지 아닌지에 대한 예측을 할 수 없다
# 따라서 두 시퀸스를 쌍(pair)으로 처리해야한다.
# 아래와 같이 두 문장을 집어넣어주면
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs
```
```python
{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
```
위와 같은 결과를 얻어볼 수 있는데, 이때 token_type_ids에서 문장 1에 해당하는 부분은 0으로 문장2에 해당하는 부분은 1로 출력되는 모습을 확인할 수 있다. 이를 통해 전체 입력(input_ids)의 어느 부분이 첫 번째 문장이고 어느 것이 두 번째 문장인지 모델에 알려주게 되는 것이다. 